In [1]:
import numpy as np
import shelve
from astropy.table import QTable
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import os
import math
import shutil
from IPython.core.debugger import Tracer
import subprocess
import re
%matplotlib inline

stable_ICs = shelve.open('stable_ICs')
ecc_b = stable_ICs['ecc_b']
ecc_c = stable_ICs['ecc_c']
semi_b = stable_ICs['semi_b']
semi_c = stable_ICs['semi_c']
dInc_b = stable_ICs['dInc_b']
dInc_c = stable_ICs['dInc_c']
dLongA_b = stable_ICs['dLongA_b']
dLongA_c = stable_ICs['dLongA_c']
dArgP_b = stable_ICs['dArgP_b']
dArgP_c = stable_ICs['dArgP_c']
mean_an_b = stable_ICs['mean_an_b']
mean_an_c = stable_ICs['mean_an_c']
good_hill = stable_ICs['good_hill']
bad_hill = stable_ICs['bad_hill']
dPrecA_b = stable_ICs['dPrecA_b']
dMass_star = stable_ICs['dMass_star']
dRadius_star = stable_ICs['dRadius_star']
dMass_b = stable_ICs['dMass_b']
dMass_c = stable_ICs['dMass_c']
dRadius_b = stable_ICs['dRadius_b']
dRadius_c = stable_ICs['dRadius_c']
dTidalQ_b = stable_ICs['dTidalQ_b']
dTidalQ_c = stable_ICs['dTidalQ_c']
dTidalQ_star = stable_ICs['dTidalQ_star']
dStopTime = stable_ICs['dStopTime']
stable_ICs.close()

In [2]:
# Parameters still need to do, by file
# b.in
name_b = 'b'
modules_b = 'eqtide distorb distrot'
sTideModel_b = 'p2'
dMass_b = dMass_b # sets to earth masses;
dRadius_b = dRadius_b # sets to earth radii
#semi_b defined above
#ecc_b defined above
bForceEqSpin_b = 1
#dRotPeriod = -1 
dObliquity_b = 45 
dRadGyra_b = 0.5
dInc_b = dInc_b #distorb
dLongA_b = dLongA_b #distorb
dArgP_b = dArgP_b #distorb
sOrbitModel_b = 'rd4' #distorb
bOutputLapl_b = 0 #distorb
dPrecA_b = dPrecA_b #distrot
bCalcDynEllip_b = 1 #distrot
dTidalQ_b = dTidalQ_b 
dK2_b = 0.3
#dMaxLockDiff = 0.1 
saTidePerts_b = 'star'
saOutputOrder_b = 'Time Semim Ecce Inc Obliquity SurfEnFluxEqtide LongP LongA PrecA' #precA only for distrot

# c.in
name_c = 'c'
modules_c = 'eqtide distorb'
sTideModel_c = 'p2'
dMass_c = dMass_c # sets to earth masses; this isnt working!!
dRadius_c = dRadius_c # sets to earth radii
#semi_c defined above
#ecc_c defined above
bForceEqSpin_c = 1
#dRotPeriod = -1 
dObliquity_c = 45 
dRadGyra_c = 0.5
dInc_c = dInc_c
dLongA_c = dLongA_c
dArgP_c = dArgP_c
sOrbitModel_c = 'rd4'
bOutputLapl_c = 0
dTidalQ_c = dTidalQ_c 
dK2_c = 0.3
#dMaxLockDiff = 0.1 
saTidePerts_c = 'star'
saOutputOrder_c = 'Time Semim Ecce Inc LongP LongA'

#star.in
name_star = 'star'
modules_star = 'eqtide'
dMass_star = dMass_star #solar masses;
dRadius_star = dMass_star * 0.0048 # * ratio of solar radii to au
dObliquity_star = 0  
bForceEqSpin_star = 1
dRotPeriod_star = -30 
dRadGyra_star = 0.5 
dTidalQ_star = dTidalQ_star
dK2_star = 0.3 
saTidePerts_star = 'b c'
saOutputOrder_s = 'Time'

#vpl.in
Sys_name = 'system'
iVerbose = 5
bOverwrite = 1
saBodyFiles = 'star.in b.in c.in'  
UnitMass = 'solar' 
UnitLength = 'aU'  
UnitTime = 'YEARS'
UnitAngle = 'd'
UnitTemp = 'K'
bDoLog = 1
iDigits = 6
dMinValue = 10**(-10)
bDoForward = 1
bVarDt = 1
dEta = 0.01
dStopTime = dStopTime
dOutputTime = 10**9


#any extra conversions for units defined in vpl.in; therefore everything needs to be in M_earth or R_earth.
print ('ICs set.')

#test edit for git update

ICs set.


In [7]:
# now to calc mutual incl
dInc_b_deg = dInc_b * u.deg
dInc_b_rad = dInc_b_deg.to(u.rad)
dInc_c_deg = dInc_c * u.deg
dInc_c_rad = dInc_c_deg.to(u.rad)
dLongA_b_deg = dLongA_b * u.deg
dLongA_b_rad = dLongA_b_deg.to(u.rad)
dLongA_c_deg = dLongA_c * u.deg
dLongA_c_rad = dLongA_c_deg.to(u.rad)

mut_incl = np.arccos(np.cos(dInc_b_rad)*np.cos(dInc_c_rad)+
                     np.sin(dInc_b_rad)*np.sin(dInc_c_rad)*np.cos(dLongA_c_rad - dLongA_b_rad))

max_idx_mut_incl = np.argmax(mut_incl)
min_idx_mut_incl = np.argmin(mut_incl)
# gives the run witb the greates mutual inclination. 

max_idx = max_idx_mut_incl
min_idx = min_idx_mut_incl
max_idx

3513

In [17]:
low_q_high_mutinc_idx = np.array([])
asc_mutincl_idx = np.argsort(mut_incl)
des_mutincl_idx = asc_mutincl_idx[::-1]
des_mutincl_idx
for idx in des_mutincl_idx:
    Q = dTidalQ_b[idx]
    if Q <= 300:
        low_q_high_mutinc_idx = np.append(low_q_high_mutinc_idx,idx)
        

In [18]:
low_q_high_mutinc_idx

array([   957.,  21897.,   3808., ...,   2697.,   9158.,   5857.])

In [21]:
dTidalQ_b[957],mut_incl[957],dMass_b[957]

(47.917443192545591, <Quantity 0.16568601994156928 rad>, -0.6032486535234578)

In [22]:
mut_incl[3513]

<Quantity 0.1690523318634666 rad>

In [4]:
t = 25000 # number of runs


n = 0
while n <= t-1: # will make n <= x amount of folders; (25000);
    # make strings of values; b.in
    name_idx = '%05i'%n
    mass_str_b = str(dMass_b[n]) 
    radius_str_b = str(dRadius_b[n])
    eqspin_str_b = str(bForceEqSpin_b)
    obl_str_b = str(dObliquity_b)
    radgy_str_b = str(dRadGyra_b)
    ecc_str_b = str(ecc_b[n])
    semi_str_b = str(semi_b[n])
    dInc_str_b = str(dInc_b[n]) # new
    dLongA_str_b = str(dLongA_b[n]) # new
    dArgP_str_b = str(dArgP_b[n]) # new
    bOutputLapl_str_b = str(bOutputLapl_b) # new
    dPrecA_str_b = str(dPrecA_b[n]) # new
    bCalcDynEllip_str_b = str(bCalcDynEllip_b)
    q_str_b = str(dTidalQ_b[n])
    dK2_str_b = str(dK2_b)
    perts_str_b = str(saTidePerts_b)
    outorder_str_b = str(saOutputOrder_b)
    
    #c.in
    mass_str_c = str(dMass_c[n]) 
    radius_str_c = str(dRadius_c[n])
    eqspin_str_c = str(bForceEqSpin_c)
    obl_str_c = str(dObliquity_c)
    radgy_str_c = str(dRadGyra_c)
    ecc_str_c = str(ecc_c[n])
    semi_str_c = str(semi_c[n])
    dInc_str_c = str(dInc_c[n])
    dLongA_str_c = str(dLongA_c[n])
    dArgP_str_c = str(dArgP_c[n])
    q_str_c = str(dTidalQ_c[n])
    dK2_str_c = str(dK2_c)
    perts_str_c = str(saTidePerts_c)
    outorder_str_c = str(saOutputOrder_c)
    
    
    #star.in
    name_star = name_star
    modules_star = modules_star
    strMass_str = str(dMass_star[n])
    strRad_str = str(dRadius_star[n])
    strObl_str = str(dObliquity_star)
    #strEqSpin_str = str(bForceEqSpin_star)
    strRotPer_str = str(dRotPeriod_star)
    strRadGyr_str = str(dRadGyra_star)
    strTidalQ_str = str(dTidalQ_star[n])
    strdK2_str = str(dK2_star)
    saTidePerts_star = saTidePerts_star
    strsaOutputOrder = saOutputOrder_s
    
    #vpl.in
    sys_name = Sys_name
    iVerbose_str = str(iVerbose)
    bOverwrite_str = str(bOverwrite)
    saBodyFiles = saBodyFiles
    UnitMass = UnitMass
    UnitLength = UnitLength
    UnitTime = UnitTime
    UnitAngle = UnitAngle
    UnitTemp = UnitTemp
    bDoLog_str = str(bDoLog)
    iDigits_str = str(iDigits)
    dMinValue_str = str(dMinValue)
    bDoForward_str = str(bDoForward)
    bVarDt_str = str(bVarDt)
    dEta_str = str(dEta)
    dStopTime_str = str(int(dStopTime[n]))
    dOutputTime_str = str(dOutputTime)
    
    
    b = open('./runs/'+name_idx+'/b.in','w')
    b_content = ('sName\t'+ name_b + 
                 '\nsaModules\t'+modules_b+
                 '\nsTideModel\t'+sTideModel_b+
                 '\n\ndMass\t'+mass_str_b+
                 '\ndRadius\t'+radius_str_b+
                 '\ndObliquity\t'+obl_str_b+
                 '\ndRadGyra\t'+radgy_str_b+
                 '\n\ndEcc\t'+ecc_str_b+
                 '\ndSemi\t'+semi_str_b+
                 '\n\ndInc\t'+dInc_str_b+
                 '\ndLongA\t'+dLongA_str_b+
                 '\ndArgP\t'+dArgP_str_b+
                 '\nsOrbitModel\t'+sOrbitModel_b+
                 '\nbOutputLapl\t'+bOutputLapl_str_b+
                 '\n\ndPrecA\t'+dPrecA_str_b+     #distrot? 
                 '\nbCalcDynEllip\t'+bCalcDynEllip_str_b+ #distrot?
                 '\n\nbForceEqSpin\t'+eqspin_str_b+
                 '\ndTidalQ\t'+q_str_b+
                 '\ndK2\t'+dK2_str_b+
                 '\nsaTidePerts\t'+perts_str_b+
                 '\n\nsaOutputOrder\t'+outorder_str_b+'\n')
    b.write(b_content)
    b.close()
    
    c = open('./runs/'+name_idx+'/c.in','w')
    c_content = ('sName\t'+ name_c + 
                 '\nsaModules\t'+modules_c+
                 '\n\ndMass\t'+mass_str_c+
                 '\ndRadius\t'+radius_str_c+
                 '\ndObliquity\t'+obl_str_c+
                 '\ndRadGyra\t'+radgy_str_c+
                 '\n\ndEcc\t'+ecc_str_c+
                 '\ndSemi\t'+semi_str_c+
                 '\n\ndInc\t'+dInc_str_c+
                 '\ndLongA\t'+dLongA_str_c+
                 '\ndArgP\t'+dArgP_str_c+
                 '\n\nbForceEqSpin\t'+eqspin_str_c+
                 '\ndTidalQ\t'+q_str_c+
                 '\ndK2\t'+dK2_str_c+
                 '\nsaTidePerts\t'+perts_str_c+
                 '\n\nsaOutputOrder\t'+outorder_str_c+'\n')
    c.write(c_content)
    c.close()
    
    star = open('./runs/'+name_idx+'/star.in','w')
    star_content = ('sName\t'+name_star+
                    '\nsaModules\t'+modules_star+
                    '\n\ndMass\t'+strMass_str+
                    '\ndRadius\t'+strRad_str+
                    '\ndObliquity\t'+strObl_str+
                    '\ndRotPeriod\t'+strRotPer_str+
                    '\ndRadGyra\t'+strRadGyr_str+
                    '\n\ndTidalQ\t'+strTidalQ_str+
                    '\ndK2\t'+strdK2_str+
                    '\n\nsaTidePerts\t'+saTidePerts_star+
                    '\n\nsaOutputOrder\t'+strsaOutputOrder+'\n')
    star.write(star_content)
    star.close()
    
    vpl = open('./runs/'+name_idx+'/vpl.in','w')
    vpl_content = ('sSystemName\t'+sys_name+
                   '\niVerbose\t'+iVerbose_str+
                   '\nbOverwrite\t'+bOverwrite_str+
                   '\n\nsaBodyFiles\t'+saBodyFiles+
                   '\n\nsUnitMass\t'+UnitMass+
                   '\nsUnitLength\t'+UnitLength+
                   '\nsUnitTime\t'+UnitTime+
                   '\nsUnitAngle\t'+UnitAngle+
                   '\nsUnitTemp\t'+UnitTemp+
                   '\n\nbDoLog\t'+bDoLog_str+
                   '\niDigits\t'+iDigits_str+
                   '\ndMinValue\t'+dMinValue_str+
                   '\n\nbDoForward\t'+bDoForward_str+
                   '\nbVarDt\t'+bVarDt_str+
                   '\ndEta\t'+dEta_str+
                   '\ndStopTime\t'+dStopTime_str+
                   '\ndOutputTime\t'+ dOutputTime_str+'\n')
    vpl.write(vpl_content)
    vpl.close()
    
    
        
 
    n += 1







print ('IC files generated')

IC files generated


In [6]:
min_idx

5857

In [23]:
max_idx

3513